In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import pandas as pd
import re
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [3]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Bidirectional

from gensim.models import Word2Vec

from keras.callbacks import EarlyStopping
from keras.models import load_model

In [4]:
!pip install tweet-preprocessor 

In [5]:
import preprocessor as p
import numpy as np 
import pandas as pd 
import emoji
import keras

import tensorflow as tf
from keras.models import Sequential



#sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#from keras.layers.recurrent import LSTM, GRU,SimpleRNN 
from keras.layers import Dense, Embedding, LSTM, GRU, Bidirectional,SimpleRNN

from keras.layers.core import Dense, Activation, Dropout
#from keras.layers.embeddings import Embedding

#importing  libraries
#Keras
import tensorflow.keras
from tensorflow.keras import regularizers, layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint



from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#import transformers
#from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
#from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tqdm import tqdm

In [6]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
## Save clean_df to csv file
#clean_df.to_csv('clean_df.csv', index=False)
Tweet_Dataset_model = pd.read_csv('Model_data.csv')

In [9]:
Tweet_Dataset_model.head()

,tweet_id,sentiment,content,content_token,synonym,clean_tweet,clean_tweet_token
0,1.956968e+09,neutral,@dannycastillo We want to trade with someone w...,"['want', 'trade', 'someone', 'houston', 'ticke...","[['privation', 'want', 'deprivation', 'needine...",want trade someone houston ticket one,"['want', 'trade', 'someone', 'houston', 'ticke..."
1,1.956969e+09,neutral,cant fall asleep,"['cant', 'fall', 'asleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant fall asleep,"['cant', 'fall', 'asleep']"
2,1.956972e+09,neutral,No Topic Maps talks at the Balisage Markup Con...,"['topic', 'map', 'talk', 'balisage', 'markup',...","[['subject', 'topic', 'theme', 'topic', 'subje...",topic map talk balisage markup conference prog...,"['topic', 'map', 'talk', 'balisage', 'markup',..."
3,1.956975e+09,neutral,@cynthia_123 i cant sleep,"['cant', 'sleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant sleep,"['cant', 'sleep']"
4,1.956976e+09,neutral,I missed the bl***y bus!!!!!!!!,"['missed', 'bly', 'bus']","[['miss', 'lose', 'miss', 'miss', 'neglect', '...",missed bl bus,"['missed', 'bl', 'bus']"


In [10]:
Tweet_Dataset_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108628 entries, 0 to 108627
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   tweet_id           8356 non-null    float64
 1   sentiment          108628 non-null  object 
 2   content            108628 non-null  object 
 3   content_token      8356 non-null    object 
 4   synonym            8356 non-null    object 
 5   clean_tweet        108391 non-null  object 
 6   clean_tweet_token  108628 non-null  object 
dtypes: float64(1), object(6)
memory usage: 5.8+ MB


In [11]:
Tweet_Dataset_model.isnull().sum()

tweet_id             100272
sentiment                 0
content                   0
content_token        100272
synonym              100272
clean_tweet             237
clean_tweet_token         0
dtype: int64

In [12]:
Tweet_Dataset_model = Tweet_Dataset_model[Tweet_Dataset_model['clean_tweet'].notna()]

In [13]:
Tweet_Dataset_model["sentiment"].value_counts()

boredom       8356
anger         8356
relief        8353
happiness     8352
fun           8352
love          8351
sadness       8349
hate          8344
enthusiasm    8343
worry         8335
surprise      8324
empty         8294
neutral       8282
Name: sentiment, dtype: int64

In [14]:
sent_to_id  = {"empty":0, "sadness":1,"enthusiasm":2,"neutral":3,"worry":4,
                        "surprise":5,"love":6,"fun":7,"hate":8,"happiness":9,"boredom":10,"relief":11,"anger":12}

In [15]:
Tweet_Dataset_model["sentiment_id"] = Tweet_Dataset_model['sentiment'].map(sent_to_id)

In [16]:
Tweet_Dataset_model.head()

,tweet_id,sentiment,content,content_token,synonym,clean_tweet,clean_tweet_token,sentiment_id
0,1.956968e+09,neutral,@dannycastillo We want to trade with someone w...,"['want', 'trade', 'someone', 'houston', 'ticke...","[['privation', 'want', 'deprivation', 'needine...",want trade someone houston ticket one,"['want', 'trade', 'someone', 'houston', 'ticke...",3
1,1.956969e+09,neutral,cant fall asleep,"['cant', 'fall', 'asleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant fall asleep,"['cant', 'fall', 'asleep']",3
2,1.956972e+09,neutral,No Topic Maps talks at the Balisage Markup Con...,"['topic', 'map', 'talk', 'balisage', 'markup',...","[['subject', 'topic', 'theme', 'topic', 'subje...",topic map talk balisage markup conference prog...,"['topic', 'map', 'talk', 'balisage', 'markup',...",3
3,1.956975e+09,neutral,@cynthia_123 i cant sleep,"['cant', 'sleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant sleep,"['cant', 'sleep']",3
4,1.956976e+09,neutral,I missed the bl***y bus!!!!!!!!,"['missed', 'bly', 'bus']","[['miss', 'lose', 'miss', 'miss', 'neglect', '...",missed bl bus,"['missed', 'bl', 'bus']",3


In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [18]:
# Start with loading all necessary libraries
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [26]:
# get most common words in training dataset
from collections import Counter 
all_words = []
for line in list(Tweet_Dataset_model['clean_tweet']):
    words = line.split()
    for word in words:
        all_words.append(word.lower())
    
    
a=Counter(all_words).most_common(10)
a

[('day', 7989),
 ('get', 7304),
 ('quot', 5405),
 ('got', 4483),
 ('good', 4375),
 ('number', 4356),
 ('im', 4227),
 ('lol', 4209),
 ('one', 4170),
 ('know', 4121)]

In [33]:
data=Tweet_Dataset_model

In [34]:
data

,tweet_id,sentiment,content,content_token,synonym,clean_tweet,clean_tweet_token,sentiment_id
0,1.956968e+09,neutral,@dannycastillo We want to trade with someone w...,"['want', 'trade', 'someone', 'houston', 'ticke...","[['privation', 'want', 'deprivation', 'needine...",want trade someone houston ticket one,"['want', 'trade', 'someone', 'houston', 'ticke...",3
1,1.956969e+09,neutral,cant fall asleep,"['cant', 'fall', 'asleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant fall asleep,"['cant', 'fall', 'asleep']",3
2,1.956972e+09,neutral,No Topic Maps talks at the Balisage Markup Con...,"['topic', 'map', 'talk', 'balisage', 'markup',...","[['subject', 'topic', 'theme', 'topic', 'subje...",topic map talk balisage markup conference prog...,"['topic', 'map', 'talk', 'balisage', 'markup',...",3
3,1.956975e+09,neutral,@cynthia_123 i cant sleep,"['cant', 'sleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant sleep,"['cant', 'sleep']",3
4,1.956976e+09,neutral,I missed the bl***y bus!!!!!!!!,"['missed', 'bly', 'bus']","[['miss', 'lose', 'miss', 'miss', 'neglect', '...",missed bl bus,"['missed', 'bl', 'bus']",3
...,...,...,...,...,...,...,...,...
108623,NaN,anger,Did a historic Jesus of_all_time exist? Im wi...,NaN,NaN,historic jesus time exist im witness hard prov...,"['historic', 'jesus', 'time', 'exist', 'im', '...",12
108624,NaN,anger,@mikeavila ack! I just interpret about your t...,NaN,NaN,ack interpret testify horrible accounting taste,"['ack', 'interpret', 'testify', 'horrible', 'a...",12
108625,NaN,anger,ne'er does anything dear cultivate and lapin ...,NaN,NaN,ne er anything dear cultivate lapin life histo...,"['ne', 'er', 'anything', 'dear', 'cultivate', ...",12
108626,NaN,anger,@mashable Link to ff &quot;how it works&quot;...,NaN,NaN,link ff quot work quot broke,"['link', 'ff', 'quot', 'work', 'quot', 'broke']",12


In [38]:
# removing usernames from tweet

data['new_tweet'] = data.content.str.replace('@', '')
data.head()

,tweet_id,sentiment,content,content_token,synonym,clean_tweet,clean_tweet_token,sentiment_id,new_tweet
0,1.956968e+09,neutral,@dannycastillo We want to trade with someone w...,"['want', 'trade', 'someone', 'houston', 'ticke...","[['privation', 'want', 'deprivation', 'needine...",want trade someone houston ticket one,"['want', 'trade', 'someone', 'houston', 'ticke...",3,dannycastillo We want to trade with someone wh...
1,1.956969e+09,neutral,cant fall asleep,"['cant', 'fall', 'asleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant fall asleep,"['cant', 'fall', 'asleep']",3,cant fall asleep
2,1.956972e+09,neutral,No Topic Maps talks at the Balisage Markup Con...,"['topic', 'map', 'talk', 'balisage', 'markup',...","[['subject', 'topic', 'theme', 'topic', 'subje...",topic map talk balisage markup conference prog...,"['topic', 'map', 'talk', 'balisage', 'markup',...",3,No Topic Maps talks at the Balisage Markup Con...
3,1.956975e+09,neutral,@cynthia_123 i cant sleep,"['cant', 'sleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant sleep,"['cant', 'sleep']",3,cynthia_123 i cant sleep
4,1.956976e+09,neutral,I missed the bl***y bus!!!!!!!!,"['missed', 'bly', 'bus']","[['miss', 'lose', 'miss', 'miss', 'neglect', '...",missed bl bus,"['missed', 'bl', 'bus']",3,I missed the bl***y bus!!!!!!!!


In [39]:
#Removing Punctuations, Numbers, and Special Characters
#[a-zA-Z] = Any single character in the range a-z or A-Z
# ^ = Start of line 
# $ = End of line 

data['new_tweet'] = data['new_tweet'].str.replace("[^a-zA-Z#]", " ")
data['new_tweet'] = data['new_tweet'].str.replace("#", "")
data.head()

,tweet_id,sentiment,content,content_token,synonym,clean_tweet,clean_tweet_token,sentiment_id,new_tweet
0,1.956968e+09,neutral,@dannycastillo We want to trade with someone w...,"['want', 'trade', 'someone', 'houston', 'ticke...","[['privation', 'want', 'deprivation', 'needine...",want trade someone houston ticket one,"['want', 'trade', 'someone', 'houston', 'ticke...",3,dannycastillo We want to trade with someone wh...
1,1.956969e+09,neutral,cant fall asleep,"['cant', 'fall', 'asleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant fall asleep,"['cant', 'fall', 'asleep']",3,cant fall asleep
2,1.956972e+09,neutral,No Topic Maps talks at the Balisage Markup Con...,"['topic', 'map', 'talk', 'balisage', 'markup',...","[['subject', 'topic', 'theme', 'topic', 'subje...",topic map talk balisage markup conference prog...,"['topic', 'map', 'talk', 'balisage', 'markup',...",3,No Topic Maps talks at the Balisage Markup Con...
3,1.956975e+09,neutral,@cynthia_123 i cant sleep,"['cant', 'sleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant sleep,"['cant', 'sleep']",3,cynthia i cant sleep
4,1.956976e+09,neutral,I missed the bl***y bus!!!!!!!!,"['missed', 'bly', 'bus']","[['miss', 'lose', 'miss', 'miss', 'neglect', '...",missed bl bus,"['missed', 'bl', 'bus']",3,I missed the bl y bus


In [40]:
# get most common words in training dataset
from collections import Counter 
all_words = []
for line in list(data['new_tweet']):
    words = line.split()
    for word in words:
        all_words.append(word.lower())
    
    
a=Counter(all_words).most_common(10)
a

[('i', 55857),
 ('to', 41046),
 ('the', 38474),
 ('a', 29495),
 ('it', 22771),
 ('my', 22267),
 ('and', 22028),
 ('you', 20468),
 ('is', 16624),
 ('for', 15894)]

In [41]:
#tokenization
data['new_tweet'] = data['new_tweet'].apply(lambda x: x.split())
data.head()

,tweet_id,sentiment,content,content_token,synonym,clean_tweet,clean_tweet_token,sentiment_id,new_tweet
0,1.956968e+09,neutral,@dannycastillo We want to trade with someone w...,"['want', 'trade', 'someone', 'houston', 'ticke...","[['privation', 'want', 'deprivation', 'needine...",want trade someone houston ticket one,"['want', 'trade', 'someone', 'houston', 'ticke...",3,"[dannycastillo, We, want, to, trade, with, som..."
1,1.956969e+09,neutral,cant fall asleep,"['cant', 'fall', 'asleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant fall asleep,"['cant', 'fall', 'asleep']",3,"[cant, fall, asleep]"
2,1.956972e+09,neutral,No Topic Maps talks at the Balisage Markup Con...,"['topic', 'map', 'talk', 'balisage', 'markup',...","[['subject', 'topic', 'theme', 'topic', 'subje...",topic map talk balisage markup conference prog...,"['topic', 'map', 'talk', 'balisage', 'markup',...",3,"[No, Topic, Maps, talks, at, the, Balisage, Ma..."
3,1.956975e+09,neutral,@cynthia_123 i cant sleep,"['cant', 'sleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant sleep,"['cant', 'sleep']",3,"[cynthia, i, cant, sleep]"
4,1.956976e+09,neutral,I missed the bl***y bus!!!!!!!!,"['missed', 'bly', 'bus']","[['miss', 'lose', 'miss', 'miss', 'neglect', '...",missed bl bus,"['missed', 'bl', 'bus']",3,"[I, missed, the, bl, y, bus]"


In [42]:
#stemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

data['new_tweet']= data['new_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
data.head()

,tweet_id,sentiment,content,content_token,synonym,clean_tweet,clean_tweet_token,sentiment_id,new_tweet
0,1.956968e+09,neutral,@dannycastillo We want to trade with someone w...,"['want', 'trade', 'someone', 'houston', 'ticke...","[['privation', 'want', 'deprivation', 'needine...",want trade someone houston ticket one,"['want', 'trade', 'someone', 'houston', 'ticke...",3,"[dannycastillo, we, want, to, trade, with, som..."
1,1.956969e+09,neutral,cant fall asleep,"['cant', 'fall', 'asleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant fall asleep,"['cant', 'fall', 'asleep']",3,"[cant, fall, asleep]"
2,1.956972e+09,neutral,No Topic Maps talks at the Balisage Markup Con...,"['topic', 'map', 'talk', 'balisage', 'markup',...","[['subject', 'topic', 'theme', 'topic', 'subje...",topic map talk balisage markup conference prog...,"['topic', 'map', 'talk', 'balisage', 'markup',...",3,"[no, topic, map, talk, at, the, balisag, marku..."
3,1.956975e+09,neutral,@cynthia_123 i cant sleep,"['cant', 'sleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant sleep,"['cant', 'sleep']",3,"[cynthia, i, cant, sleep]"
4,1.956976e+09,neutral,I missed the bl***y bus!!!!!!!!,"['missed', 'bly', 'bus']","[['miss', 'lose', 'miss', 'miss', 'neglect', '...",missed bl bus,"['missed', 'bl', 'bus']",3,"[i, miss, the, bl, y, bus]"


In [43]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fatma.er\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

#stopwords = set(stopwords.words('english'))
stopwords = nltk.corpus.stopwords.words('english')

In [45]:
newStopWords = ['u','go','got','via','or','ur','us','in','i','let','the','to','is','amp','make','one','day','days','get']
stopwords.extend(newStopWords)

In [46]:
import string 

def process(text):
    # Check characters to see if they are in punctuation
    nopunc = set(char for char in list(text) if char not in string.punctuation)
    # Join the characters to form the string.
    nopunc = " ".join(nopunc)
    # remove any stopwords if present
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords]
    

In [47]:
data['new_tweet'] = data['new_tweet'].apply(process) 
data.head()

,tweet_id,sentiment,content,content_token,synonym,clean_tweet,clean_tweet_token,sentiment_id,new_tweet
0,1.956968e+09,neutral,@dannycastillo We want to trade with someone w...,"['want', 'trade', 'someone', 'houston', 'ticke...","[['privation', 'want', 'deprivation', 'needine...",want trade someone houston ticket one,"['want', 'trade', 'someone', 'houston', 'ticke...",3,"[trade, ticket, dannycastillo, someon, want, h..."
1,1.956969e+09,neutral,cant fall asleep,"['cant', 'fall', 'asleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant fall asleep,"['cant', 'fall', 'asleep']",3,"[cant, fall, asleep]"
2,1.956972e+09,neutral,No Topic Maps talks at the Balisage Markup Con...,"['topic', 'map', 'talk', 'balisage', 'markup',...","[['subject', 'topic', 'theme', 'topic', 'subje...",topic map talk balisage markup conference prog...,"['topic', 'map', 'talk', 'balisage', 'markup',...",3,"[ml, topic, onlin, balisag, topicmap, bobdc, t..."
3,1.956975e+09,neutral,@cynthia_123 i cant sleep,"['cant', 'sleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant sleep,"['cant', 'sleep']",3,"[cant, cynthia, sleep]"
4,1.956976e+09,neutral,I missed the bl***y bus!!!!!!!!,"['missed', 'bly', 'bus']","[['miss', 'lose', 'miss', 'miss', 'neglect', '...",missed bl bus,"['missed', 'bl', 'bus']",3,"[bl, bus, miss]"


In [49]:
def string (text):
    to_return=""
    for i in list(text):
        to_return += str(i) + " "
    to_return = to_return[:-1]
    
    return to_return
    
       
data['new_tweet'] = data['new_tweet'].apply(string)
data.head() 

,tweet_id,sentiment,content,content_token,synonym,clean_tweet,clean_tweet_token,sentiment_id,new_tweet
0,1.956968e+09,neutral,@dannycastillo We want to trade with someone w...,"['want', 'trade', 'someone', 'houston', 'ticke...","[['privation', 'want', 'deprivation', 'needine...",want trade someone houston ticket one,"['want', 'trade', 'someone', 'houston', 'ticke...",3,trade ticket dannycastillo someon want houston
1,1.956969e+09,neutral,cant fall asleep,"['cant', 'fall', 'asleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant fall asleep,"['cant', 'fall', 'asleep']",3,cant fall asleep
2,1.956972e+09,neutral,No Topic Maps talks at the Balisage Markup Con...,"['topic', 'map', 'talk', 'balisage', 'markup',...","[['subject', 'topic', 'theme', 'topic', 'subje...",topic map talk balisage markup conference prog...,"['topic', 'map', 'talk', 'balisage', 'markup',...",3,ml topic onlin balisag topicmap bobdc talk pro...
3,1.956975e+09,neutral,@cynthia_123 i cant sleep,"['cant', 'sleep']","[['buzzword', 'cant', 'bank', 'cant', 'camber'...",cant sleep,"['cant', 'sleep']",3,cant cynthia sleep
4,1.956976e+09,neutral,I missed the bl***y bus!!!!!!!!,"['missed', 'bly', 'bus']","[['miss', 'lose', 'miss', 'miss', 'neglect', '...",missed bl bus,"['missed', 'bl', 'bus']",3,bl bus miss


In [53]:
#Bag of words

In [54]:
#Split data into training and testing sets 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data["new_tweet"], 
                                                    data["sentiment_id"], test_size = 0.2, random_state = 42)

print("training set :",x_train.shape,y_train.shape)
print("testing set :",x_test.shape,y_test.shape)

training set : (86712,) (86712,)
testing set : (21679,) (21679,)


In [55]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)

In [56]:
x_train_counts = count_vect.fit_transform(x_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

print(x_train_counts.shape)
print(x_train_tfidf.shape)

(86712, 39890)
(86712, 39890)


In [57]:
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)

print(x_test_counts.shape)
print(x_test_tfidf.shape)

(21679, 39890)
(21679, 39890)


# Model building  Random Forest Classifier

In [58]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)
model.fit(x_train_tfidf,y_train)

RandomForestClassifier(n_estimators=200)

In [59]:
predictions = model.predict(x_test_tfidf)

In [60]:
#Accuracy_score
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)*100

71.99132801328474

In [61]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,predictions)

array([[1557,   11,   11,   31,   21,   17,    8,   11,   15,    6,    8,
           7,    0],
       [  40,  718,   23,  232,  276,   59,   57,   62,   69,   72,   19,
          35,    4],
       [  12,   11, 1480,   18,    8,   11,   16,    3,   14,   11,    4,
           5,    0],
       [  41,  125,   24,  737,  262,   56,  100,   50,   35,  184,   11,
          51,    8],
       [  27,  257,   23,  437,  502,   55,   76,   44,   61,  152,   12,
          39,   10],
       [  33,   50,   20,   98,   54, 1279,   38,   34,   28,   44,    4,
          31,    3],
       [  21,   47,   24,  133,   71,   56, 1126,   40,   38,  141,    3,
          28,    0],
       [  18,   18,   18,   76,   35,   46,   37, 1379,   30,   45,    4,
          22,    2],
       [  20,   14,   13,   47,   27,   28,   12,   13, 1403,    5,    8,
          12,    1],
       [  18,   31,   22,  205,   93,   56,  144,   86,   23,  880,    3,
          65,    4],
       [   0,    4,    0,    7,    1,    2,    0, 

In [64]:
predictions

array([12,  0,  8, ...,  0,  6,  9], dtype=int64)

# Logistic Regression

In [65]:
#https://github.com/Shrneha/Twitter-sentiment-analysis/blob/master/Twitter.ipynb

In [66]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(random_state=400 )
logmodel.fit(x_train_tfidf,y_train)

LogisticRegression(random_state=400)

In [67]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=400, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

LogisticRegression(multi_class='warn', random_state=400, solver='warn')

In [68]:
log_predictions = logmodel.predict(x_test_tfidf)

In [69]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,log_predictions)

array([[1393,   34,   36,   39,   34,   15,   14,   11,   26,   27,   21,
          46,    7],
       [  82,  585,   60,  151,  263,   75,   55,   78,  124,   66,   42,
          68,   17],
       [  14,   19, 1384,   24,   20,   20,   19,   21,   18,   18,    9,
          25,    2],
       [  69,  119,   51,  580,  251,   76,   97,   97,   59,  159,   22,
          77,   27],
       [  50,  230,   59,  264,  566,   86,   54,   79,   82,  120,   28,
          54,   23],
       [  54,   82,   57,  106,   83,  903,   78,   88,   70,  100,   25,
          59,   11],
       [  40,   46,   60,  112,   68,   53,  889,  102,   34,  215,   13,
          90,    6],
       [  35,   38,   44,   60,   43,   61,   64, 1154,   36,  113,   11,
          64,    7],
       [  33,   49,   21,   22,   47,   26,   12,   33, 1288,    8,   21,
          35,    8],
       [  29,   38,   70,  194,   87,   77,  169,  160,   25,  647,   17,
         108,    9],
       [   6,    8,    1,    0,    6,    5,    3, 

In [72]:
#f1_score(y_test,log_predictions)

In [73]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,log_predictions)*100

63.014899211218236

# Gradient Boosting Regressor

In [74]:
from sklearn.ensemble import GradientBoostingRegressor

alg= GradientBoostingRegressor(n_estimators= 550, learning_rate= 0.1, max_depth= 3)
alg.fit(x_train_tfidf,y_train)

GradientBoostingRegressor(n_estimators=550)

In [75]:
alg_predictions = logmodel.predict(x_test_tfidf)

In [76]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,alg_predictions)

array([[1393,   34,   36,   39,   34,   15,   14,   11,   26,   27,   21,
          46,    7],
       [  82,  585,   60,  151,  263,   75,   55,   78,  124,   66,   42,
          68,   17],
       [  14,   19, 1384,   24,   20,   20,   19,   21,   18,   18,    9,
          25,    2],
       [  69,  119,   51,  580,  251,   76,   97,   97,   59,  159,   22,
          77,   27],
       [  50,  230,   59,  264,  566,   86,   54,   79,   82,  120,   28,
          54,   23],
       [  54,   82,   57,  106,   83,  903,   78,   88,   70,  100,   25,
          59,   11],
       [  40,   46,   60,  112,   68,   53,  889,  102,   34,  215,   13,
          90,    6],
       [  35,   38,   44,   60,   43,   61,   64, 1154,   36,  113,   11,
          64,    7],
       [  33,   49,   21,   22,   47,   26,   12,   33, 1288,    8,   21,
          35,    8],
       [  29,   38,   70,  194,   87,   77,  169,  160,   25,  647,   17,
         108,    9],
       [   6,    8,    1,    0,    6,    5,    3, 

In [77]:
#f1_score(y_test,alg_predictions)

In [78]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,alg_predictions)*100

63.014899211218236